## Stack
Stack is a `LIFO`, Last In First Out data structure. It is a list with restriction that insertion and deletion occurs only at one end.  
Stack can be implemented using arrays or using linked list.

### Array based implementation

In [ ]:
class SimpleArrayStack<T> {
    private final int MAX_SIZE;
    private final Object[] elements;

    private int top = -1;

    public SimpleArrayStack(int size) {
        MAX_SIZE = size;
        elements = new Object[MAX_SIZE];
    }

    public boolean isEmpty() {
        return top == -1;
    }

    public void push(T x) {
        if (top < MAX_SIZE - 1) {
            elements[++top] = x;
        } else {
            throw new IllegalArgumentException("Stack is full");
        }
    }

    public T pop() {
        if (isEmpty()) {
            throw new IllegalArgumentException("Queue is empty");
        }

        top--;
        return (T) elements[top + 1];
    }
}

We can modify the above code to extend the array when the array completely fills up. In such case the time complexities are:
1. push : Best case $O(1)$, Worst case (when we have to extend) $O(n)$, Average case $O(1)$
2. pop : $O(1)$

In [ ]:
class ResizableArrayStack<T> {
    private int arraySize = 5;
    private Object[] elements = new Object[arraySize];

    private int top = -1;

    public boolean isEmpty() {
        return top == -1;
    }

    public void push(T x) {
        if (top == arraySize - 1) {
            resize(2 * arraySize);
        }

        elements[++top] = x;
    }

    public T pop() {
        if (isEmpty()) {
            throw new IllegalArgumentException("Stack is empty");
        } else if (arraySize > 3 * top) {
            resize(arraySize / 2);
        }

        top--;
        return (T) elements[top + 1];
    }

    private void resize(int newSize) {
        Object[] newElements = new Object[newSize];

        for (int i = 0; i < elements.length; i++) {
            newElements[i] = elements[i];
        }

        elements = newElements;
        arraySize = newSize;
    }
}

### A Note on Time Complexity of Insertion of Elements in a Resizable Array
Whenever the array is full we double the size of array, copy the elements from old array to the new one. So what is the time complexity? To calculate the time complexity of addition of element into a resizable array, we do *[amortized time complexity](https://brilliant.org/wiki/amortized-analysis/)* which is basically weighted average time complexity.
  
Suppose we have to insert $n$ elements into an array and we start with array having size $1$.  
So as soon as we add $1$ element, we create a new array with double the size and copy that $1$ element to the new array. When we add one more element, we double array and copy over $2$ elements. Adding the third element takes constant time. So for $n$ elements, the times are: 

$$1 \rightarrow 1$$
$$2 \rightarrow 2 = 2^0 + 1$$
$$3 \rightarrow 3 = 2^1 + 1$$
$$4 \rightarrow 1$$
$$5 \rightarrow 5 = 2^2 + 1$$
$$6 \rightarrow 1$$
$$\ldots$$

<img src="images/amortized_analysis.png" />

### Linked List Based Implementation

In [ ]:
class LinkedListStack<T> {
    private class Node {
        public T data;
        public Node next;

        public Node(T data) {
            this.data = data;
        }
    }

    private Node top;

    public boolean isEmpty() {
        return top == null;
    }

    public void push(T x) {
        Node temp = new Node(x);
        temp.next = top;
        top = temp;
    }

    public T pop() {
        if (isEmpty()) {
            throw new IllegalArgumentException("Stack is empty");
        }

        T value = top.data;
        top = top.next;
        return value;
    }
}

Linked List based stack implementation has the same push and pop time complexity as array based implementation (as long as array based implementation has fixed length).

### Java Implementation
We can use either of `Stack`, `ArrayDeque` or `ArrayList`.

**Stack:** a `Vector` based implementation
- Instantiate : `Stack<Integer> stack = new Stack<>();`
- Size: `stack.size();`
- Push: `stack.push(5)`
- Pop: `int item = stack.pop()`
- Iterate: `for(Integer element: stack)`

**ArrayDeque:** prefer this.
- Instantiate : `ArrayDeque<Integer> stack = new ArrayDeque<>();`
- Size: `stack.size();`
- Push: `stack.push(5)`, `stack.addFirst(5)`
- Pop: `int item = stack.pop()` or `int item = stack.removeFirst()`
- Iterate: `for(Integer element: stack)`

**ArrayList:**
- Instantiate : `ArrayList<Integer> stack = new ArrayList<>();`
- Size: `stack.size();`
- Push: `stack.add(5)`
- Pop: `int item = stack.remove(stack.size() - 1)`
- Iterate: `for(Integer element: stack)`

## Infix Postfix and Prefix
A normal arithmetic expression comprises of opertor in between operands. Expression is evaluated from left to right and we have to take into consideration the precedence of various operators. Paranthesis is used to specify what is evaluated first. The expression `A + B * C + D` can be rewritten as `((A + (B * C)) + D)` to show that the multiplication happens first, followed by the leftmost addition.  

**Prefix:** notation requires that all operators precede the two operands that they work on  
**Postfix:** notation requires that its operators come after the corresponding operands  

Examples:
```
| Infix             | Prefix        | Postfix       |
|-------------------|---------------|---------------|
| A + B * C + D     | + + A * B C D | A B C * + D + |
| (A + B) * (C + D) | * + A B + C D | A B + C D + * |
| A * B + C * D     | + * A B * C D | A B * C D * + |
```

Prefix and postfix notations have the benefit of not requiring paranthesis to determine the order of operations.

**Converting infix to postfix and prefix:** if the infix operation is completely paranthesized, we traverse the expression and a) in case of postfix replace ')' with the operator b) in case of prefix replace '(' with the operator.

![Postfix](https://runestone.academy/runestone/books/published/pythonds/_images/moveright.png)
![Prefix](https://runestone.academy/runestone/books/published/pythonds/_images/moveleft.png)  

Infix to postfix:
```java
public static String convert(String input) {
    StringBuffer output = new StringBuffer();

    // Only operators are added to this stack
    Stack<String> operatorStack = new Stack<>();

    // Considering input has space separated symbols
    String[] tokens = input.split(" ");

    for(String token: tokens) {
        // if not operator, just append to output
        if(!(token.equals("+") || token.equals("-") ||
            token.equals("*") || token.equals("/") ||
            token.equals("(") || token.equals(")"))) {
            output.append(token);
        // add opening paranthesis to the stack
        } else if(token.equals("(")) {
            operatorStack.push("(");
        // on encountering closing paranthesis, remove
        // everything from stack (and append) till you
        // get to corresponding opening paranthesis
        } else if(token.equals(")")) {
            String popped = null;
            while(!operatorStack.isEmpty() && 
                  (popped = operatorStack.pop()) != "(") {
                output.append(popped);
            }
        // For other operators before inserting into stack
        // remove all operators which have equal or greater
        // precedence
        } else if(token.equals("+") || token.equals("-")) {
            while(!operatorStack.isEmpty()) {
                String top = operatorStack.peek();
                if(top.equals("+") || top.equals("-") ||
                   top.equals("*") || top.equals("/")) {
                    output.append(operatorStack.pop());
                } else {
                    break;
                }
            }
            operatorStack.push(token);
        } else if(token.equals("*") || token.equals("/")) {
            while(!operatorStack.isEmpty()) {
                String top = operatorStack.peek();
                if(top.equals("*") || top.equals("/")) {
                    output.append(operatorStack.pop());
                } else {
                    break;
                }
            }
            operatorStack.push(token);
        }
    }

    // Empty out stack
    while(!operatorStack.isEmpty()) {
        output.append(operatorStack.pop());
    }

    return new String(output);
}
```

Evaluating postfix expression
```java
public static int evaluate(String input) {
    // Only numbers are added to this stack
    Stack<Integer> numStack = new Stack<>();

    // Considering input has space separated symbols
    String[] tokens = input.trim().split(" ");

    for(String token: tokens) {
        // push numbers to stack
        if(!(token.equals("+") || token.equals("-") ||
             token.equals("*") || token.equals("/"))) {
            numStack.push(Integer.parseInt(token));
        } else {
            int rightOperand = numStack.pop();
            int leftOperand = numStack.pop();
            int result = 0;

            // evaluate binary expression as per operator
            // then push result to stack again
            if(token.equals("+")) {
                result = leftOperand + rightOperand;
            } else if(token.equals("-")) {
                result = leftOperand - rightOperand;
            } else if(token.equals("*")) {
                result = leftOperand * rightOperand;
            } else if(token.equals("/")) {
                result = leftOperand / rightOperand;
            }

            numStack.push(result);
        }
    }

    return numStack.pop();
}
```

**Q 1:** Implement a function `getMin()` which returns the minimum element in the stack.  
**Answer:** So get minimum element of stack, we need to make use of two stacks. One main stack and the other stack which holds current minimum at the top.

In [6]:
stack = []
# This stack contains all the minimum
# elements
min_stack = []
def push(x):
    stack.append(x)
    # Both stacks are empty, so append
    if len(stack) == 0:
        min_stack.append(x)
    else:
        # Check if the newly added element is less
        # than the current minimum element
        if x <= min_stack[-1]:
            min_stack.append(x)
            
def pop():
    popped = stack.pop()
    if popped == min_stack[-1]:
        min_stack.pop()
        
def get_min():
    return min_stack[-1]
        
push(7)
push(8)
push(3)
print(get_min())
push(1)
push(0)
push(0)
pop()
print(get_min())

3
0


How can we maintain minimum without using extra space? If all the numbers to be inserted into the stack are positive then we can follow the below steps. Consider the array is `6, 3, 7, 8, 2`. We insert 6 first into the stack. The variable `min_stack = 6`. Next when we push 3, `min_stack = 3`, however the value inserted into the stack will be `3 - min_stack` which is `3 - 6 = -3`. We will insert 7 and 8 as it is. When we insert 2, we make `min_stack = 2`, however we insert `2 - 3 = -1` into the stack. And so on. The stack now is `6, -3, 7, 8, -1`.  
Now when we pop elements and it is negative, then to get to the original number we need to follow the following equation: `x - prev_min_stack = top_of_stack`. But how to get the previous min from current min? `prev_min_stack = min_stack - top_of_stack`. `prev_min_stack = 2 - (-1) = 3`. So the popped elements is `x - 3 = -1 = 2`. Make `min_stack = prev_min_stack`.

**Q 2:** Sort a stack  
**Answer:**

In [2]:
stack = []

def sort():
    # Create aux stack
    aux = []
        
    while(len(stack) != 0):
        popped = stack.pop()
        # If aux is empty just push
        if len(aux) == 0:
            aux.append(popped)
            
        # If the element to be added to aux is greater
        # than or equal to element at top, push
        elif popped <= aux[-1]:
            aux.append(popped)
        # Pop out out elements greater before pushing
        else:
            while(len(aux) != 0 and aux[-1] < popped):
                stack.append(aux.pop())
            aux.append(popped)
            
    while(len(aux) != 0):
        stack.append(aux.pop())
            
stack = [2,3,1,5,1]
sort()
print(stack)

[1, 1, 2, 3, 5]


**Q 3:** Given a string, remove all the consequtive duplicate elements. For example, if the string is `abbck`, then return `ack`. However consider `kabbal`. It will convert to `kaal` and then finally to `kl`. Finally if we pass `aaa` as input, we should get `a` .  
**Answer:**

In [4]:
def remove_duplicate(input):
    stack = []
    for i in input:
        if len(stack) == 0:
            stack.append(i)
        else:
            # If the element to be inserted is already
            # present at the top, pop that element
            if stack[-1] == i:
                stack.pop()
            # Else push element to the stack
            else:
                stack.append(i)
    return ''.join(stack)

print(remove_duplicate('kabbbal'))
print(remove_duplicate('kabbal'))
print(remove_duplicate('aaa'))
print(remove_duplicate('abbck'))

kabal
kl
a
ack


**Q 4:** Determine whether the given set of paranthesis is valid or not. Paranthesis can be either of the following: `[, {, (, ), }, ]`  
**Answer:**

In [21]:
def valid_paranthesis(input):
    opening = ['{', '[', '(']
    closing = ['}', ']', ')']
    stack = []
    for i in input:
        if i in closing and len(stack) == 0:
            return False
        
        if i in opening:
            stack.append(i)
        else:
            popped = stack.pop()
            if (i == '}' and popped == '{') or (i == ']' and popped == '[') or (i == ')' and popped == '('):
                continue
            else:
                return False
            
    if len(stack) == 0:
        return True
    else:
        return False

par = '{[)(]}'                
print(valid_paranthesis(par))

par = '{[()]}'                
print(valid_paranthesis(par))

False
True


**Q 5:** Given an array, for each element find the closest number (to the right) greater than that number. For example, if the array is `[10, 7, 4, 2, 9, 10, 11, 3, 2]`, the answer would be `[11, 9, 9, 9, 10, 11, -1, -1, -1]` .   
**Answer:** 

In [8]:
def next_larger(A):
    i = len(A) - 1
    stack = []
    answer = [-1] * len(A)
    while(i >= 0):
        val = A[i]
        
        while(len(stack) != 0 and val >= stack[-1]):
            stack.pop()
            
        if len(stack) != 0:
            answer[i] = stack[-1]
        
        stack.append(val)
        i -= 1
    return answer

A = [10, 7, 4, 2, 9, 10, 11, 3, 2]
print(next_larger(A))

[11, 9, 9, 9, 10, 11, -1, -1, -1]


Similarly, previous larger would be

In [11]:
def prev_larger(A):
    i = 0
    stack = []
    answer = [-1] * len(A)
    while(i < len(A)):
        val = A[i]
        
        while(len(stack) != 0 and val >= stack[-1]):
            stack.pop()
            
        if len(stack) != 0:
            answer[i] = stack[-1]
        
        stack.append(val)
        i += 1
    return answer

A = [10, 7, 4, 2, 9, 10, 11, 3, 2]
print(prev_larger(A))

[-1, 10, 7, 4, 10, -1, -1, 11, 3]


In similar fashion,

In [12]:
def next_smaller(A):
    i = len(A) - 1
    stack = []
    answer = [-1] * len(A)
    while(i >= 0):
        val = A[i]
        
        while(len(stack) != 0 and val <= stack[-1]):
            stack.pop()
            
        if len(stack) != 0:
            answer[i] = stack[-1]
        
        stack.append(val)
        i -= 1
    return answer

A = [10, 7, 4, 2, 9, 10, 11, 3, 2]
print(next_smaller(A))

[7, 4, 2, -1, 3, 3, 3, 2, -1]


And,

In [13]:
def prev_smaller(A):
    i = 0
    stack = []
    answer = [-1] * len(A)
    while(i < len(A)):
        val = A[i]
        
        while(len(stack) != 0 and val <= stack[-1]):
            stack.pop()
            
        if len(stack) != 0:
            answer[i] = stack[-1]
        
        stack.append(val)
        i += 1
    return answer

A = [10, 7, 4, 2, 9, 10, 11, 3, 2]
print(prev_smaller(A))

[-1, -1, -1, -1, 2, 9, 10, 2, -1]


**Q 6:** Implement UNDO and REDO operations using stacks.  
**Answer:** Maintain two stack say UNDO stack and REDO stack. We use UNDO stack to keep history of all the operations that we have done. Whenever a user does UNDO operation, we simply pop the top of the element from UNDO stack and push it to the REDO stack. Then, if the user wants to do REDO operation we pop the top of the element of REDO stack and push it to UNDO. If the user performs a new operation, then we make the stack REDO empty.

**Q 7:** Insert an element at the bottom of the stack.  
**Answer:** We can solve this by maintaining an auxiliary stack.

In [1]:
def insert_at_bottom(stack, x):
    if len(stack) == 0:
        stack.append(x)
        return
    
    aux = []
    while(len(stack) != 0):
        aux.append(stack.pop())
    
    stack.append(x)
    while(len(aux) != 0):
        stack.append(aux.pop())
        
stack = [4,3,2]
insert_at_bottom(stack, 1)
print(stack)

[1, 4, 3, 2]


Or we can use recursion.

In [2]:
def insert_at_bottom(stack, x):
    if len(stack) == 0:
        stack.append(x)
        return
    
    popped = stack.pop()
    insert_at_bottom(stack, x)
    stack.append(popped)
    
stack = [4,3,2]
insert_at_bottom(stack, 1)
print(stack)

[1, 4, 3, 2]


**Q 8:** Reverse a stack.  
**Answer:** For every stack element, insert it at the bottom. Time complexity is $O(n^2)$. Space complexity is $O(n)$

In [3]:
def reverse_stack(stack):
    # Copy all elements to an aux stack
    aux = []
    while(len(stack) != 0):
        aux.append(stack.pop())
        
    while(len(aux) != 0):
        insert_at_bottom(stack, aux.pop())
        
stack = [1,2,3,4]
reverse_stack(stack)
print(stack)

[4, 3, 2, 1]


**Q 9:** Given an array, return the number of subarrays where the leftmost element is lesser than the rest of the elements. For example, if the array is `[1,4,2,5,3]`, then all possible subarrays are: `[1]`, `[4]`, `[2]`, `[5]`, `[3]`, `[1,4]`, `[2,5]`, `[1,4,2]`, `[2,5,3]`, `[4,2,5]`, `[1,4,2,5]`, `[1,4,2,5,3]`. So the count is 12.  
**Answer:** This problem can be solved iterating over each element and finding out how many elments are greater than it towards the right. This solution is $O(n^2)$.

In [1]:
def leftmost_smaller(A):
    count = 0
    for i in range(len(A)):
        c = 1
        for j in range(i+1, len(A)):
            if A[j] > A[i]:
                c += 1
        
        count += c        
    return count

print(leftmost_smaller([1,4,2,5,3]))

12


Another way to approach is to think recursively:

In [4]:
# visited contains (start, end) indices
# of the visited subarray
def leftmost_smaller(A, start, end, visited):
    # this subarray has been already accounted for
    # return count as zero
    if (start, end) in visited:
        return 0
    
    visited.add((start, end))
    if start > end:
        return 0
    
    if start == end:
        return 1
    
    count = 0
    if(A[start] < A[end]):
        count += 1
        
    return count + leftmost_smaller(A, start + 1, end, visited) + leftmost_smaller(A, start, end - 1, visited)

print(leftmost_smaller([1,4,2,5,3], 0, 4, set()))

12


**Q 10** Given a $N \times 2$ array, where if `A[i][0] == 1` then add `A[i][1]` to stack and return -1. If `A[i][0] == 2` then remove the most frequent item from the stack. If the most frequent item in the stack has more than one candidate, then return the one which is closest to the top.  
**Answer** To solve this we can maintain multiple stacks, each for a particular frequency

In [4]:
def max_freq_stack(A):
    freq_map = {}
    stacks = []
    
    # We create our own definition of push and pop
    def push(val):
        # Get current frequency
        current_freq = freq_map.get(val, 0)
        updated_freq = current_freq + 1
        freq_map[val] = updated_freq
        if len(stacks) < updated_freq:
            stacks.append([])
            stacks[updated_freq - 1].append(val)
        else:
            stacks[updated_freq - 1].append(val)
        return -1
            
    def pop():
        i = len(stacks) - 1
        while i >= 0:
            if len(stacks[i]) != 0:
                popped = stacks[i].pop()
                freq_map[popped] -= 1
                return popped
            else:
                i -= 1
    
    output = []
    for a in A:
        if a[0] == 1:
            output.append(push(a[1]))
        elif a[0] == 2:
            output.append(pop())
            
    return output

A = [[1, 5], [1, 7], [1, 5], [1, 7], [1, 4], [1, 5], [2, 0], [2, 0], [2, 0], [2, 0]]
print(max_freq_stack(A))

[-1, -1, -1, -1, -1, -1, 5, 7, 5, 4]


**Q 11** Given a string A representing an absolute path for a file (Unix-style). Return the string A after simplifying the absolute path. For example: `/a/./b/../../c/` is converted to `/c`  
**Answer** . We can remove the last character if it is a `/`.

In [11]:
def simplify_directory(path):
    if path[-1] == '/':
        path = path[:-1]

    # Split path based on '/'
    path = path.split('/')
    stack = []
    i = 0
    while i < len(path):
        if path[i] == '.' or path[i] == '':
            # do nothing
            pass
        elif path[i] == '..':
            if len(stack) != 0:
                stack.pop()
        else:
            stack.append(path[i])
        i += 1

    answer = ''
    while len(stack) != 0:
        answer = '/' + stack.pop() + answer

    return answer


path = '/a/./b/../../c/'
print(simplify_directory(path))

path = '/home/'
print(simplify_directory(path))

/c
/home
